In [1]:
import pandas as pd
import numpy as np

import torch 
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim

In [2]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [3]:
x_test = test_df.values/255
y = train_df.label.values
x = train_df.iloc[:,1:].values/255
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.2) 

In [4]:
trainTorch_x = torch.from_numpy(train_x).type(torch.FloatTensor)
trainTorch_y = torch.from_numpy(train_y).type(torch.LongTensor)


valTorch_x = torch.from_numpy(val_x).type(torch.FloatTensor)
valTorch_y = torch.from_numpy(val_y).type(torch.LongTensor) 

testTorch_x = torch.from_numpy(np.array(x_test)).type(torch.FloatTensor)
# testTorch_x = testTorch_x.to(device)

In [5]:
train = torch.utils.data.TensorDataset(trainTorch_x, trainTorch_y)
val = torch.utils.data.TensorDataset(valTorch_x, valTorch_y)

In [6]:
train_loader = DataLoader(train, batch_size = 100, shuffle = False)
val_loader = DataLoader(val, batch_size = 100, shuffle = False)

In [7]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = torch.nn.Conv2d(1,6,kernel_size = 5,padding = 2)
        self.conv2 = torch.nn.Conv2d(6,16,kernel_size = 5)
        self.fc1 = torch.nn.Linear(16*5*5,120)
        self.fc2 = torch.nn.Linear(120,84)
        self.fc3 = torch.nn.Linear(84,10)
        
    def forward(self,x):
        batch_size = x.size(0)
#         x为张量，张量.size 取出维度  取0  得到就是样本数量 n 1 28 28
        x = x.view(batch_size,1,28,28)
        x = F.max_pool2d( F.relu(self.conv1(x)) , 2)
        x = F.max_pool2d( F.relu(self.conv2(x)) , 2)
        
        x = x.view(batch_size,-1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
model = Net()
# model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.5)

In [9]:
# 将单轮循环封装为一个函数

def train_func(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
#         inputs,target = inputs.to(device),target.to(device)
        optimizer.zero_grad()
        
        
#         forward and backward and update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss: %.3f' % (epoch + 1,batch_idx + 1,running_loss / 300))
            running_loss =0.0
            

In [10]:
def test_func():
    correct = 0
    total = 0
    with torch.no_grad():
#         执行之后在下面代码就不会执行梯度
        for data in val_loader:
            images,labels = data
#             images,labels = images.to(device),labels.to(device)
#             拿数据
            outputs = model(images)
#             做预测，拿到的结果是一个矩阵，每一行都是一个独热向量
            _, predicted = torch.max(outputs.data,dim = 1)
#           返回 最大值 和 每一行的最大值下标
#           指定沿着维度1（往下 行是第0个维度，向右 列是第一个维度）
            total += labels.size(0)
#             label是一个N 1元组 size 取 0 就是？

            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [12]:
def predict_func():
    with torch.no_grad():
        outputs = model(testTorch_x)
        index, predicted = torch.max(outputs.data,dim = 1)
    ans2 = predicted
    submission_file2 = pd.read_csv('./sample_submission.csv')
    submission_file2.Label = ans2
    submission_file2.to_csv('submission_cnn_lenet.csv', index=False)
    print("saved")

In [14]:
if __name__ == '__main__':
    for epoch in range(99):
        train_func(epoch)
        test_func()
    predict_func()

[1,  300] loss: 1.266
Accuracy on test set: 84 %
[2,  300] loss: 0.373
Accuracy on test set: 90 %
[3,  300] loss: 0.263
Accuracy on test set: 92 %
[4,  300] loss: 0.204
Accuracy on test set: 94 %
[5,  300] loss: 0.163
Accuracy on test set: 95 %
[6,  300] loss: 0.136
Accuracy on test set: 95 %
[7,  300] loss: 0.118
Accuracy on test set: 96 %
[8,  300] loss: 0.105
Accuracy on test set: 96 %
[9,  300] loss: 0.095
Accuracy on test set: 96 %
[10,  300] loss: 0.087
Accuracy on test set: 96 %
[11,  300] loss: 0.081
Accuracy on test set: 97 %
[12,  300] loss: 0.075
Accuracy on test set: 97 %
[13,  300] loss: 0.071
Accuracy on test set: 97 %
[14,  300] loss: 0.066
Accuracy on test set: 97 %
[15,  300] loss: 0.062
Accuracy on test set: 97 %
[16,  300] loss: 0.059
Accuracy on test set: 97 %
[17,  300] loss: 0.056
Accuracy on test set: 97 %
[18,  300] loss: 0.053
Accuracy on test set: 97 %
[19,  300] loss: 0.050
Accuracy on test set: 97 %
[20,  300] loss: 0.048
Accuracy on test set: 97 %
[21,  300